In [1]:
import pandas as pd
from sklearn.utils import resample

df = pd.read_csv('Creditcard_data.csv')

df_majority = df[df.Class == 0]
df_minority = df[df.Class == 1]

df_minority_upsampled = resample(df_minority,replace=True,n_samples=len(df_majority),random_state=42)

df_balanced = pd.concat([df_majority, df_minority_upsampled])

df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("Balanced Class distribution:")
print(df_balanced['Class'].value_counts())

df_balanced.to_csv('Balanced_Creditcard_data.csv', index=False)

Balanced Class distribution:
Class
1    763
0    763
Name: count, dtype: int64


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

In [5]:
df = pd.read_csv('Creditcard_data.csv')

In [7]:
df_majority = df[df.Class == 0]
df_minority = df[df.Class == 1]

df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority),
                                 random_state=42)

df_balanced = pd.concat([df_majority, df_minority_upsampled])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [8]:
n_size = 385

samples = {}

samples['Sample 1'] = df_balanced.sample(n=n_size, random_state=1)


samples['Sample 2'] = df_balanced.groupby('Class', group_keys=False).apply(lambda x: x.sample(int(n_size/2), random_state=2))

k = len(df_balanced) // n_size
start = 0
samples['Sample 3'] = df_balanced.iloc[start::k].head(n_size)

df_balanced['cluster'] = np.repeat(range(10), len(df_balanced)//10 + 1)[:len(df_balanced)]
selected_clusters = np.random.choice(range(10), size=5, replace=False)
samples['Sample 4'] = df_balanced[df_balanced['cluster'].isin(selected_clusters)].drop(columns=['cluster'])


samples['Sample 5'] = df_balanced.sample(n=400, random_state=5, replace=True)


/tmp/ipython-input-3205235418.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  samples['Sample 2'] = df_balanced.groupby('Class', group_keys=False).apply(lambda x: x.sample(int(n_size/2), random_state=2))


In [9]:
models = {
    'M1: Logistic Regression': LogisticRegression(max_iter=2000),
    'M2: Decision Tree': DecisionTreeClassifier(random_state=42),
    'M3: Random Forest': RandomForestClassifier(random_state=42),
    'M4: SVM': SVC(random_state=42),
    'M5: KNN': KNeighborsClassifier()
}

In [10]:
results = pd.DataFrame(index=models.keys(), columns=samples.keys())

for sample_name, sample_data in samples.items():

    if 'cluster' in sample_data.columns:
        sample_data = sample_data.drop(columns=['cluster'])

    X = sample_data.drop('Class', axis=1)
    y = sample_data['Class']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for model_name, model in models.items():

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)

        results.loc[model_name, sample_name] = round(acc, 4)

print("\n--- Model Accuracy Results per Sample ---")
print(results)

results.to_csv('Model_Comparison_Results.csv')


--- Model Accuracy Results per Sample ---
                        Sample 1 Sample 2 Sample 3 Sample 4 Sample 5
M1: Logistic Regression    0.961    0.961   0.8961   0.9085    0.925
M2: Decision Tree         0.9481    0.974    0.974      1.0   0.9875
M3: Random Forest            1.0      1.0    0.987      1.0      1.0
M4: SVM                   0.7532   0.6494   0.7662   0.6732    0.625
M5: KNN                    0.974   0.9221   0.9091   0.9412    0.975
